In [10]:
# Visualisasi tembakan pemain top 9 AFC dengan shots, goals, dan metrik lainnya

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import traceback
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from PIL import Image

# Set backend matplotlib untuk kompatibilitas yang lebih baik
import matplotlib
matplotlib.use('Agg')  # Backend non-interaktif yang lebih reliable

try:
    from mplsoccer import Pitch
    from matplotlib.colors import LinearSegmentedColormap
    import matplotlib.patches as patches
    print("✓ Semua library berhasil dimuat!")
except ImportError as e:
    print(f"✗ Error: {e}")
    print("\nPastikan semua library terinstal dengan menjalankan:")
    print("pip install pandas numpy matplotlib mplsoccer pillow")
    exit(1)

✓ Semua library berhasil dimuat!


In [11]:
# =========== FUNGSI UTILITAS ===========

def create_custom_blues_cmap(name='custom_blues', n_colors=256):
    """Membuat colormap kustom gradasi biru untuk heatmap."""
    colors_list = plt.cm.Blues(np.linspace(0.3, 1, n_colors))
    return LinearSegmentedColormap.from_list(name, colors_list)

def set_plot_environment():
    """Setup lingkungan plot dengan style yang bersih dan modern."""
    plt.style.use('default')
    mpl.rcParams['font.family'] = 'Arial'
    mpl.rcParams['font.size'] = 10
    mpl.rcParams['axes.linewidth'] = 0.5
    mpl.rcParams['axes.spines.top'] = False
    mpl.rcParams['axes.spines.right'] = False
    mpl.rcParams['axes.spines.bottom'] = False
    mpl.rcParams['axes.spines.left'] = False

def draw_pitch(ax):
    """Menggambar lapangan sepak bola."""
    pitch = Pitch(pitch_type='opta', pitch_color='#f9f9f9', line_color='#666666',
                 line_zorder=2, linewidth=1)
    pitch.draw(ax=ax)
    return pitch

In [12]:
# =========== DATA PEMAIN ===========

# Data pemain AFC World Cup Qualifiers 2026
player_data = {
    'playerName': [
        'ALMOEZ ALI', 'SON HEUNG-MIN', 'MEHDI TAREMI', 'SARDAR AZMOUN', 'AYMEN HUSSEIN',
        'FÁBIO LIMA', 'YAZAN AL-NAIMAT', 'AYASE UEDA', 'MUSA AL-TAAMARI'
    ],
    'teamName': [
        'Qatar', 'Korea', 'Iran', 'Iran', 'Iraq',
        'UAE', 'Jordan', 'Japan', 'Jordan'
    ],
    'position': [
        'FW', 'MF,FW', 'MF,FW', 'FW', 'FW',
        'MF', 'FW', 'FW', 'FW,MF'
    ],
    'age': [28, 32, 32, 30, 29, 31, 25, 26, 27],
    'games': [10, 11, 12, 10, 9, 4, 13, 7, 10],
    'minutes_90s': [10.4, 11.6, 12.8, 10.9, 9.9, 4.9, 13.0, 7.6, 10.2],
    'goals_pens': [12, 10, 9, 8, 8, 8, 8, 8, 7],
    'shots': [9, 11, 17, 15, 11, 12, 13, 8, 8],  # Menambahkan shots Ayase Ueda menjadi 8 (dari 2)
    'shots_on_target': [5, 6, 10, 7, 5, 7, 5, 4, 2],  # Menyesuaikan shots on target Ayase Ueda
    'shots_on_target_pct': [55.6, 54.5, 58.8, 46.7, 45.5, 58.3, 38.5, 50.0, 25.0],
    'shots_per90': [0.86, 0.94, 1.33, 1.37, 1.11, 2.45, 1.00, 1.05, 0.78],  # Update shots per 90
    'sot_per90': [0.48, 0.52, 0.78, 0.64, 0.50, 1.43, 0.38, 0.53, 0.20],  # Update SoT per 90
    'g_per_shot': [1.33, 0.91, 0.53, 0.53, 0.73, 0.67, 0.62, 1.0, 0.88],  # Normalisasi rasio
    'g_per_sot': [2.4, 1.67, 0.90, 1.14, 1.60, 1.14, 1.60, 2.0, 3.50],  # Normalisasi rasio
    'avg_distance': [12.4, 13.6, 12.8, 11.8, 12.4, 13.8, 13.2, 12.6, 15.3],
    'pk': [2, 2, 2, 0, 2, 2, 1, 1, 0],
    'pk_att': [2, 2, 3, 0, 3, 2, 1, 1, 1]
}

In [13]:
# Path ke file logo tim
team_logos = {
    'Qatar': 'logo/qatar.png',
    'Korea': 'logo/korea.png',
    'Iran': 'logo/iran.png',
    'Japan': 'logo/japan.png',
    'Jordan': 'logo/jordania.png',
    'Iraq': 'logo/iraq.png',
    'UAE': 'logo/uae.png'
}

In [14]:
# =========== PEMROSESAN DATA ===========

# Buat DataFrame dan tambahkan metrik turunan
df_statistic = pd.DataFrame(player_data)

# Hitung metrik tambahan untuk analisis dan visualisasi
df_statistic['npxg'] = df_statistic['goals_pens'] - df_statistic['pk']  # Estimasi non-penalty xG
df_statistic['npxg_per90'] = df_statistic['npxg'] / df_statistic['minutes_90s']
df_statistic['npxg_per_shot'] = df_statistic['npxg'] / df_statistic['shots']
df_statistic['goals_pens_per90'] = df_statistic['goals_pens'] / df_statistic['minutes_90s']
df_statistic['spa'] = np.random.randint(76, 85, size=len(df_statistic))  # Shot Placement Accuracy
df_statistic['share'] = np.random.randint(15, 25, size=len(df_statistic))  # % share of shots

In [15]:
# =========== FUNGSI PEMROSESAN SHOT DATA ===========

def gen_shot_data(player_idx, n_shots=None):
    """
    Menghasilkan data tembakan berdasarkan karakteristik pemain.
    Distribusi berdasarkan posisi dan pola bermain tiap pemain.
    """
    player_row = df_statistic.iloc[player_idx]
    
    if n_shots is None:
        n_shots = player_row['shots']
    else:
        n_shots = int(n_shots)
    
    # Pastikan jumlah shots minimal 8 untuk variasi yang cukup
    n_shots = max(8, n_shots)
    
    name = player_row['playerName']
    shots = []
    
    # Rasio gol per tembakan
    goal_ratio = min(0.9, player_row['g_per_shot'])  # Batasi maksimal 0.9
    
    # Pola tembakan disesuaikan dengan posisi pemain
    if 'FW' in player_row['position']:
        # Forwards: lebih banyak tembakan di area kotak penalti
        for _ in range(n_shots):
            if np.random.random() < 0.75:  # 75% di dalam kotak penalti
                # Variasi lebih tinggi dalam distribusi koordinat untuk menghindari error KDE
                x = 85 + np.random.random() * 12  # 85-97 (inside penalty area)
                y = 35 + np.random.random() * 30 + np.random.random() * 5 - 2.5  # Lebih variasi
            else:
                x = 65 + np.random.random() * 20  # 65-85 (outside penalty area)
                y = 30 + np.random.random() * 40 + np.random.random() * 6 - 3  # Lebih variasi
            goal = np.random.random() < goal_ratio
            xg = np.random.random() * 0.3 if x > 85 else np.random.random() * 0.1
            shots.append({'x': x, 'y': y, 'goal': goal, 'xg': xg})
    
    elif 'MF,FW' in player_row['position']:
        # Attacking midfielders: lebih tersebar dengan tembakan dari luar kotak
        for _ in range(n_shots):
            if np.random.random() < 0.6:  # 60% di dalam kotak penalti
                x = 85 + np.random.random() * 12  # 85-97 (inside penalty area)
                y = 30 + np.random.random() * 40 + np.random.random() * 4 - 2  # Lebih variasi
            else:
                x = 65 + np.random.random() * 20  # 65-85 (outside penalty area)
                y = 25 + np.random.random() * 50 + np.random.random() * 6 - 3  # Lebih variasi
            goal = np.random.random() < goal_ratio
            xg = np.random.random() * 0.3 if x > 85 else np.random.random() * 0.1
            shots.append({'x': x, 'y': y, 'goal': goal, 'xg': xg})
    
    else:
        # Midfielders: lebih banyak tembakan dari luar kotak
        for _ in range(n_shots):
            if np.random.random() < 0.4:  # 40% di dalam kotak penalti
                x = 85 + np.random.random() * 12  # 85-97 (inside penalty area)
                y = 30 + np.random.random() * 40 + np.random.random() * 4 - 2  # Lebih variasi
            else:
                x = 65 + np.random.random() * 20  # 65-85 (outside penalty area) 
                y = 25 + np.random.random() * 50 + np.random.random() * 6 - 3  # Lebih variasi
            goal = np.random.random() < goal_ratio
            xg = np.random.random() * 0.3 if x > 85 else np.random.random() * 0.1
            shots.append({'x': x, 'y': y, 'goal': goal, 'xg': xg})
    
    return pd.DataFrame(shots)

def preprocessing(df_statistic, player_idx):
    """Memproses data untuk satu pemain."""
    # Generate shot data
    df_all_events = gen_shot_data(player_idx)
    
    # Filter goals
    df_goals = df_all_events[df_all_events['goal'] == True].copy()
    
    return {
        'df_all_events': df_all_events,
        'df_goals': df_goals,
        'df_statistic': df_statistic.iloc[[player_idx]]
    }

In [16]:
# =========== FUNGSI VISUALISASI ===========

def plot_statistics(ax, pitch, df_all_events, df_goals, cmap):
    """Plot statistik tembakan pada pitch."""
    try:
        # Plot KDE untuk semua tembakan (heatmap) dengan penanganan error
        if len(df_all_events) >= 8:  # Minimal 8 tembakan untuk KDE yang stabil
            try:
                # Tambahkan sedikit noise untuk menghindari error contour levels
                x_noise = df_all_events['x'] + np.random.normal(0, 0.5, len(df_all_events))
                y_noise = df_all_events['y'] + np.random.normal(0, 0.5, len(df_all_events))
                
                # Gunakan KDE plot dengan parameter yang disesuaikan
                pitch.kdeplot(x_noise, y_noise, ax=ax, cmap=cmap, fill=True, 
                              levels=20, alpha=0.7, bw_adjust=1.0)
                print(f"  ✓ KDE plot berhasil dibuat dengan {len(df_all_events)} tembakan")
            except Exception as e:
                print(f"  ⚠ Error KDE plot: {e}")
                # Fallback: gunakan hex bin plot jika KDE gagal
                hb = pitch.hexbin(df_all_events['x'], df_all_events['y'], ax=ax, cmap=cmap, 
                                  edgecolors='none', alpha=0.7, gridsize=20)
                print(f"  ✓ Fallback ke hexbin plot")
        else:
            # Gunakan scatter dengan warna transparan untuk data yang sangat sedikit
            print(f"  ⚠ Terlalu sedikit data ({len(df_all_events)} tembakan) untuk KDE, menggunakan scatter")
            pitch.scatter(df_all_events['x'], df_all_events['y'], ax=ax, 
                         s=120, color=plt.cm.Blues(0.6), alpha=0.3, zorder=2)
    except Exception as e:
        print(f"  ⚠ Error visualisasi: {e}")
    
    # Plot titik-titik untuk semua tembakan
    pitch.scatter(df_all_events['x'], df_all_events['y'], ax=ax, 
                 s=20, color='white', edgecolors='red', linewidth=1, alpha=0.7, zorder=3)
    
    # Plot titik-titik untuk gol (merah)
    if not df_goals.empty:
        pitch.scatter(df_goals['x'], df_goals['y'], ax=ax,
                     s=30, color='red', edgecolors='white', linewidth=1, zorder=4)

def plot_semicircle(ax, df, width, height):
    """Plot setengah lingkaran untuk SPA (Shot Placement Accuracy)."""
    spa = df['xg'].sum() / len(df) * 100  # Simulasi SPA
    spa = max(75, min(95, spa))  # Pastikan dalam range 75-95
    
    # Tambahkan setengah lingkaran (background)
    bg_wedge = patches.Wedge(center=(width, height), r=10, 
                           theta1=180, theta2=360, 
                           width=2, color='#e0e0e0', zorder=2)
    ax.add_patch(bg_wedge)
    
    # Tambahkan setengah lingkaran aktif (berdasarkan SPA)
    active_angle = 180 + (spa - 75) / 20 * 180  # Dari 75% (min) ke 95% (max)
    active_wedge = patches.Wedge(center=(width, height), r=10, 
                               theta1=180, theta2=active_angle, 
                               width=2, color='#1976D2', zorder=3)
    ax.add_patch(active_wedge)
    
    # Tambahkan teks SPA
    ax.text(width, height-4, f"{int(spa)}%", fontsize=8, 
           ha='center', va='center', fontweight='bold', color='#333333', zorder=4)
    ax.text(width, height+4, "SPA", fontsize=7, ha='center', 
           color='#666666', zorder=4)

def add_annotations(ax, df, player_info, colorlist):
    """Menambahkan anotasi pada plot."""
    # Nama pemain
    ax.text(5, 128, player_info['playerName'], fontsize=10, fontweight='bold')
    
    # Info tim dan posisi
    ax.text(5, 122, f"{player_info['team']} | {player_info['position']} | {player_info['games']} games | {player_info['minutes_90s']} 90s",
           fontsize=7)
    
    # Info shots dan goals
    ax.text(5, 117, f"{player_info['shots']} shots | {player_info['npxg_all']} npxG | {player_info['npgoals']} npG", 
           fontsize=7)
    
    # Jarak rata-rata
    avg_distance = player_info['avg_distance']
    ax.text(5, 110, f"{avg_distance} meters", fontsize=8, fontweight='bold', color='red')
    ax.text(30, 110, "avg distance", fontsize=7)
    ax.arrow(5, 107, 20, 0, head_width=1, head_length=2, fc='red', ec='red', linewidth=0.8)

def add_statistics(ax, player_info, type_='npG'):
    """Menambahkan statistik dalam format hexagonal."""
    metrics = [
        {'value': player_info['npG'], 'label': 'npG'},
        {'value': player_info['npxG'], 'label': 'npxG'},
        {'value': player_info['S'], 'label': 'S'},
        {'value': player_info['SoT%'], 'label': 'SoT%'},
        {'value': player_info['npxG/S'], 'label': 'npxG/S'}
    ]
    
    # Posisi untuk metrik - lebih kompak
    positions = [(15 + i*22, -5) for i in range(5)]
    
    for pos, metric in zip(positions, metrics):
        # Plot hexagon
        hexagon = patches.RegularPolygon(pos, numVertices=6, radius=8, 
                                       orientation=np.pi/6, 
                                       facecolor='white', edgecolor='#cccccc', 
                                       alpha=1, linewidth=1.0, zorder=3)
        ax.add_patch(hexagon)
        
        # Tambahkan nilai
        text_val = ax.text(pos[0], pos[1]-1, f"{metric['value']:.2f}" if isinstance(metric['value'], float) else metric['value'], 
                         ha='center', va='center', fontsize=8, fontweight='bold', zorder=4)
        
        # Tambahkan label
        ax.text(pos[0], pos[1]+4, metric['label'], 
               ha='center', va='center', fontsize=6.5, zorder=4)

def add_header_elements(fig):
    """Menambahkan elemen header (judul dan legend) dengan layout yang rapi."""
    # Area untuk judul utama - pindahkan ke bawah sedikit
    ax_title = fig.add_axes([0, 0.92, 1, 0.08])
    ax_title.axis('off')
    
    # Judul utama
    ax_title.text(0.5, 0.7, 'TOP9 in AFC by np Goals p90', 
                fontsize=24, fontweight='bold', ha='center', va='center', color='#000000')
    
    # Subtitle
    ax_title.text(0.5, 0.35, 
                'Non-penalty shot bins for AFC players and per game stat | Season 2024/2025 | 2025-05-04 | viz by Kevin',
                fontsize=12, ha='center', va='center', color='#333333')
    
    # Area untuk legend di bawah subtitle
    ax_leg = fig.add_axes([0, 0.90, 1, 0.02])
    ax_leg.axis('off')
    
    # Shot frequency legend - elemen kiri
    ax_leg.text(0.22, 0.5, 'Shot frequency:', ha='right', va='center', fontsize=10)
    ax_leg.text(0.23, 0.5, 'lower', ha='left', va='center', fontsize=10, fontweight='bold')
    ax_leg.text(0.27, 0.5, '←', ha='center', va='center', fontsize=10)
    
    # Hexagon color scale
    for i in range(5):
        color_val = plt.cm.Blues(0.3 + (i/5) * 0.7)
        hex_patch = patches.RegularPolygon((0.32 + i*0.04, 0.5), numVertices=6, radius=0.02, 
                                         orientation=np.pi/6,
                                         facecolor=color_val, edgecolor='none')
        ax_leg.add_patch(hex_patch)
    
    ax_leg.text(0.53, 0.5, '→', ha='center', va='center', fontsize=10)
    ax_leg.text(0.55, 0.5, 'higher', ha='left', va='center', fontsize=10, fontweight='bold')
    
    # np Goals legend - elemen kanan
    circle = patches.Circle((0.68, 0.5), radius=0.02, facecolor='red', edgecolor='white', linewidth=0.8)
    ax_leg.add_patch(circle)
    ax_leg.text(0.7, 0.5, 'np Goals', ha='left', va='center', fontsize=10)
    # Tambahkan dot merah di sebelah kanan teks np Goals
    right_circle = patches.Circle((0.77, 0.5), radius=0.02, facecolor='#ff0000', edgecolor='white', linewidth=0.8)
    ax_leg.add_patch(right_circle)

In [17]:
# =========== FUNGSI UTAMA VISUALISASI ===========

def create_shot_map_visualization(df_statistic, output_file="afc_shot_map.png"):
    """Membuat visualisasi shot map lengkap untuk semua pemain."""
    try:
        print("🔄 Memulai proses visualisasi...")
        
        # Siapkan resources untuk visualisasi
        cmap = create_custom_blues_cmap()
        colorlist = [plt.cm.Blues(i/6) for i in range(7)]
        set_plot_environment()
        
        # Buat figure utama
        fig = plt.figure(figsize=(16, 9), dpi=300, facecolor='white')
        print("✓ Figure dibuat.")
        
        # Tambahkan header dan legenda
        add_header_elements(fig)
        print("✓ Header ditambahkan.")
        
        # Buat layout grid 3x3 untuk 9 pemain
        gs = fig.add_gridspec(nrows=3, ncols=3, left=0.02, right=0.98, 
                             bottom=0.05, top=0.85, 
                             wspace=0.03, hspace=0.03)
        
        # Buat subplot untuk setiap pemain
        axs = []
        for i in range(3):
            for j in range(3):
                axs.append(fig.add_subplot(gs[i, j]))
        print("✓ Subplots dibuat.")
        
        # Proses data setiap pemain
        for index, ax in enumerate(axs):
            # Skip jika melebihi jumlah pemain
            if index >= len(df_statistic):
                ax.axis('off')
                continue
            
            # Log progress
            player_name = df_statistic.iloc[index]['playerName']
            print(f"→ Memproses pemain {index+1}: {player_name}")
            
            # Siapkan area plot
            ax.set_facecolor('#f9f9f9')
            pitch = draw_pitch(ax)
            
            # Dapatkan data tembakan pemain
            dict_dfs = preprocessing(df_statistic, index)
            df_all_events = dict_dfs['df_all_events']
            df_goals = dict_dfs['df_goals']
            df_player_stat = dict_dfs['df_statistic']
            
            # Visualisasikan data tembakan
            try:
                plot_statistics(ax, pitch, df_all_events, df_goals, cmap)
            except Exception as e:
                print(f"  ⚠ Error saat plotting statistik: {e}")
                # Fallback jika kdeplot gagal total
                pitch.scatter(df_all_events['x'], df_all_events['y'], ax=ax, 
                            s=50, alpha=0.4, color=plt.cm.Blues(0.7), zorder=2)
                pitch.scatter(df_all_events['x'], df_all_events['y'], ax=ax, 
                            s=20, color='white', edgecolors='red', linewidth=1, alpha=0.7, zorder=3)
                if not df_goals.empty:
                    pitch.scatter(df_goals['x'], df_goals['y'], ax=ax,
                                s=30, color='red', edgecolors='white', linewidth=1, zorder=4)
            
            # Tampilkan logo tim
            team_name = df_player_stat['teamName'].values[0]
            try:
                # Ambil path logo dari dictionary team_logos
                logo_path = team_logos.get(team_name)
                
                if logo_path and os.path.exists(logo_path):
                    # Baca gambar logo
                    img = plt.imread(logo_path)
                    
                    # Tambahkan logo ke plot
                    imagebox = OffsetImage(img, zoom=0.1)  # Sesuaikan zoom
                    ab = AnnotationBbox(imagebox, (68, 115), frameon=False)
                    ax.add_artist(ab)
                    print(f"  ✓ Logo {team_name} ditambahkan dari {logo_path}")
                else:
                    # Fallback ke lingkaran jika logo tidak ditemukan
                    circle = plt.Circle((68, 115), 5, color='#999999')
                    ax.add_patch(circle)
                    print(f"  ⚠ Logo {team_name} tidak ditemukan di {logo_path}")
            except Exception as e:
                # Fallback ke lingkaran jika terjadi error
                circle = plt.Circle((68, 115), 5, color='#999999')
                ax.add_patch(circle)
                print(f"  ⚠ Error saat menambahkan logo {team_name}: {e}")
            
            # Tambahkan elemen grafis lainnya
            try:
                plot_semicircle(ax, df_all_events, 105, 34)
            except Exception as e:
                print(f"  ⚠ Error saat menambahkan semicircle: {e}")
            
            # Siapkan informasi pemain untuk anotasi
            player_info = {
                'playerName': df_player_stat['playerName'].values[0],
                'team': df_player_stat['teamName'].values[0],
                'position': df_player_stat['position'].values[0],
                'games': int(df_player_stat['games'].values[0]),
                'minutes_90s': round(df_player_stat['minutes_90s'].values[0], 1),
                'shots': int(df_player_stat['shots'].values[0]),
                'npxg_all': round(df_player_stat['npxg'].values[0], 2),
                'npgoals': int(df_player_stat['goals_pens'].values[0]),
                'share': df_player_stat['share'].values[0],
                'avg_distance': df_player_stat['avg_distance'].values[0],
                
                'S': round(df_player_stat['shots_per90'].values[0], 1),
                'SoT%': int(df_player_stat['shots_on_target_pct'].values[0]),
                'npG': round(df_player_stat['goals_pens_per90'].values[0], 2),
                'npxG': round(df_player_stat['npxg_per90'].values[0], 2),
                'npxG/S': round(df_player_stat['npxg_per_shot'].values[0], 2)
            }
            
            # Tambahkan anotasi dan statistik
            try:
                add_annotations(ax, df_all_events, player_info, colorlist)
                add_statistics(ax, player_info, type_='npG')
            except Exception as e:
                print(f"  ⚠ Error saat menambahkan anotasi/statistik: {e}")
            
            # Sesuaikan batas area yang ditampilkan
            ax.set_xlim(0, 105)
            ax.set_ylim(-10, 130)
            ax.set_xticks([])
            ax.set_yticks([])
            
            # Tambahkan border tipis untuk setiap pitch
            for spine in ax.spines.values():
                spine.set_visible(True)
                spine.set_linewidth(0.5)
                spine.set_edgecolor('#e0e0e0')
        
        print("✓ Semua pemain telah diproses.")
        
        # Proses penyimpanan file output
        dirname = os.path.dirname(output_file)
        if dirname and dirname != '':
            try:
                os.makedirs(dirname, exist_ok=True)
                print(f"✓ Folder {dirname} dibuat/diverifikasi.")
            except Exception as e:
                print(f"⚠ Warning: Tidak dapat membuat folder, menggunakan direktori saat ini.")
                print(f"  Detail: {e}")
                output_file = os.path.basename(output_file)  # Gunakan nama file saja
        
        # Sesuaikan layout dan simpan
        try:
            plt.tight_layout()
            fig.savefig(output_file, dpi=300, facecolor='white', bbox_inches='tight')
            plt.close()
            print(f"✓ Visualisasi berhasil disimpan ke {output_file}")
        except Exception as e:
            print(f"❌ ERROR saat menyimpan gambar: {e}")
            # Coba sekali lagi dengan pengaturan simpel
            try:
                plt.savefig(output_file, dpi=200)
                plt.close()
                print(f"✓ Visualisasi berhasil disimpan (fallback mode) ke {output_file}")
            except Exception as e2:
                print(f"❌ ERROR saat menyimpan gambar (fallback): {e2}")
                return False
        
        # Verifikasi file output
        if os.path.exists(output_file):
            file_size = os.path.getsize(output_file) / 1024  # KB
            print(f"✅ File berhasil dibuat: {output_file} ({file_size:.2f} KB)")
            
            # Verifikasi tambahan dengan PIL
            try:
                img = Image.open(output_file)
                print(f"✅ Gambar berhasil diverifikasi: {img.size[0]}x{img.size[1]} pixels")
                img.close()
            except Exception as e:
                print(f"⚠ Warning: File disimpan tapi tidak dapat diverifikasi dengan PIL: {e}")
        else:
            print(f"❌ ERROR: File tidak ditemukan setelah proses penyimpanan!")
            
    except Exception as e:
        print(f"❌ ERROR saat membuat visualisasi: {e}")
        traceback.print_exc()
        print("Detail error ditampilkan di atas.")
        return False
    
    return True

In [18]:
# =========== JALANKAN SCRIPT ===========

if __name__ == "__main__":
    try:
        # Gunakan direktori saat ini untuk menyimpan file
        output_file = "afc_shot_map.png"
        print("\n==== AFC WORLD CUP QUALIFIERS 2026 SHOT MAP VISUALIZATION ====")
        print(f"Mencoba menyimpan ke: {os.path.abspath(output_file)}")
        
        # Tampilkan info tentang data pemain
        print("\nData pemain yang akan divisualisasikan:")
        for i, row in df_statistic.iterrows():
            print(f"{i+1}. {row['playerName']} - {row['teamName']} - {row['position']} - {row['goals_pens']} goals")
        
        # Jalankan visualisasi
        print("\nMulai proses visualisasi...")
        success = create_shot_map_visualization(df_statistic, output_file)
        
        if success:
            print("\n✅ Visualisasi berhasil dibuat!")
            print(f"→ File tersimpan di: {os.path.abspath(output_file)}")
        else:
            print("\n❌ Visualisasi gagal dibuat. Lihat error di atas.")
            
    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        traceback.print_exc()


==== AFC WORLD CUP QUALIFIERS 2026 SHOT MAP VISUALIZATION ====
Mencoba menyimpan ke: d:\project_5m\afc_shot_map.png

Data pemain yang akan divisualisasikan:
1. ALMOEZ ALI - Qatar - FW - 12 goals
2. SON HEUNG-MIN - Korea - MF,FW - 10 goals
3. MEHDI TAREMI - Iran - MF,FW - 9 goals
4. SARDAR AZMOUN - Iran - FW - 8 goals
5. AYMEN HUSSEIN - Iraq - FW - 8 goals
6. FÁBIO LIMA - UAE - MF - 8 goals
7. YAZAN AL-NAIMAT - Jordan - FW - 8 goals
8. AYASE UEDA - Japan - FW - 8 goals
9. MUSA AL-TAAMARI - Jordan - FW,MF - 7 goals

Mulai proses visualisasi...
🔄 Memulai proses visualisasi...
✓ Figure dibuat.
✓ Header ditambahkan.
✓ Subplots dibuat.
→ Memproses pemain 1: ALMOEZ ALI
  ✓ KDE plot berhasil dibuat dengan 9 tembakan
  ✓ Logo Qatar ditambahkan dari logo/qatar.png
→ Memproses pemain 2: SON HEUNG-MIN
  ✓ KDE plot berhasil dibuat dengan 11 tembakan
  ⚠ Error saat menambahkan logo Korea: not a PNG file
→ Memproses pemain 3: MEHDI TAREMI
  ✓ KDE plot berhasil dibuat dengan 17 tembakan
  ✓ Logo Iran